Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_best_guarantee_delta_nonzero">_best_guarantee_delta_nonzero</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import b, m, theta, defaults
from proveit.logic import NotEquals, InSet
from proveit.numbers import (
        zero, one, two, pi, Abs, Add, Complex, frac, IntervalOC, Less, greater, Mult, Exp)
# QPE common exprs
from proveit.physics.quantum.QPE import _b_round, _delta_b_round, _t, _two_pow_t
 # QPE theorems
from proveit.physics.quantum.QPE import (
        _alpha_m_mod_as_geometric_sum, _alpha_are_complex, _best_round_is_int,
        _delta_b_is_real, _delta_b_is_zero_or_non_int,
        _phase_from_best_with_delta_b, _scaled_delta_b_round_in_interval,
        _t_in_natural_pos, _two_pow_t_is_nat_pos)
from proveit.trigonometry import Sin

In [ ]:
%proving _best_guarantee_delta_nonzero

In [ ]:
defaults.assumptions = [_best_guarantee_delta_nonzero.antecedent]

Keep the factor of $2$ from combining with $2^t$ in the steps below:

In [ ]:
Exp.change_simplification_directives(factor_numeric_rational=True)

In [ ]:
alpha_index = _best_guarantee_delta_nonzero.consequent.lhs.base.operand.index

## Start from $\alpha_{m~\textrm{mod}~2^t}$ expressed as a geometric sum, instantiated with $m:b_r$

In [ ]:
_alpha_m_mod_as_geometric_sum

In [ ]:
_alpha_eq_01 = (
        _alpha_m_mod_as_geometric_sum.instantiate({m: _b_round}))

### Put this in terms of $\delta_{b_r}$

In [ ]:
_phase_from_best_with_delta_b

In [ ]:
_phase_from_best_with_delta_b_inst = (
        _phase_from_best_with_delta_b.instantiate({b: _b_round}))

In [ ]:
# We want to know that _delta_b_round and 2^t are real so that
# auto-simplification will manifest in the substitute() step further below
_delta_b_is_real

In [ ]:
_delta_b_is_real.instantiate({b: _b_round})

In [ ]:
with Exp.temporary_simplification_directives() as tmp_directives:
    tmp_directives.reduce_double_exponent = False # don't simplify k double-exponent
    _alpha_eq_02 = (
        _alpha_eq_01.inner_expr().rhs.factors[1].summand.base.exponent.
        factors[3].operands[0].substitute(_phase_from_best_with_delta_b_inst.rhs))

## Evaluate the Finite Geometric Sum

To evaluate the summation as a finite geometric sum, we need to establish that the constant ratio $e^{2\pi i \delta_{b_r}} \neq 1$ which is automatically proven given $\delta_{b_r} \notin \mathbb{Z}$.

In [ ]:
_delta_b_is_zero_or_non_int

In [ ]:
_delta_b_is_zero_or_non_int_inst = _delta_b_is_zero_or_non_int.instantiate({b: _b_round})

In [ ]:
_delta_b_round_non_int = _delta_b_is_zero_or_non_int_inst.derive_right_if_not_left()

We can then evaluate the sum using the standard formula for a finite geometric sum:

In [ ]:
_alpha_eq_03 = _alpha_eq_02.inner_expr().rhs.factors[1].geom_sum_reduce()

## Take the Magnitude (Abs) of Both Sides

### Pre-requisites to allow auto-simplification to go through

Establish that the $\alpha$ expression is in the Complex numbers.

In [ ]:
_alpha_are_complex.instantiate({m:alpha_index})

Prove that $\pi |\delta_{b_r}| \in \left(0, \frac{\pi}{2}\right)$ which is important to ensure the denominator is non-zero.  Along the way, we prove $2^t \pi |\delta_{b_r}| \leq \frac{\pi}{2}$ which will be useful for bounding the numerator.

In [ ]:
angle = Mult(pi, Abs(_delta_b_round))

In [ ]:
Less(zero, angle).prove()

In [ ]:
scaled_abs_bound = (Abs(_scaled_delta_b_round_in_interval.element).deduce_weak_upper_bound(frac(one, two))
         .inner_expr().lhs.simplify())

Close enough to $2^t \pi |\delta_{b_r}| \leq \frac{\pi}{2}$ (the translation occurs automatically via canonical forms):

In [ ]:
two_pow_t_times_angle__bound = scaled_abs_bound.left_mult_both_sides(pi)

In [ ]:
two_pow_t_bound = _two_pow_t.deduce_bound(_t_in_natural_pos.derive_element_lower_bound())

In [ ]:
two_pow_t_times_angle__bound.lhs.deduce_bound(greater(_two_pow_t, one))

In [ ]:
Less(angle, frac(pi, two)).prove()

### Now take the absolute value on each side 

In [ ]:
with Mult.temporary_simplification_directives() as tmp_directives:
    # pull the 2^t to the front
    tmp_directives.order_key_fn = lambda factor : 0 if factor==_two_pow_t else 1
    _alpha_eq_04 = _alpha_eq_03.abs_both_sides()

## Bound the numerator and the denominator

### Bound the sin function in the _denominator_ utilizing $\sin{\theta} < \theta$ for $\theta > 0$

In [ ]:
denominator_sin_bound = _alpha_eq_04.rhs.factors[1].denominator.deduce_linear_upper_bound()

### Bound the sin function in the _numerator_ utilizing $\sin{\theta} \ge \frac{2}{\pi}(\theta)$ for $0 < \theta \le \frac{\pi}{2}$

In [ ]:
numerator_sin_bound = _alpha_eq_04.rhs.factors[1].numerator.deduce_linear_lower_bound()

### Apply these numerator and denominator bounds

In [ ]:
rhs_bound = _alpha_eq_04.rhs.deduce_bound([numerator_sin_bound, denominator_sin_bound])

In [ ]:
_alpha_ineq = _alpha_eq_04.apply_transitivity(rhs_bound)

## Finally square both sides

In [ ]:
_alpha_ineq.square_both_sides()

In [ ]:
%qed